In [38]:
### Predict Messages
from langchain.chat_models import ChatOpenAI
from langchain.schema  import HumanMessage, AIMessage, SystemMessage
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]) ## call Chat large language models API.

messages  = [
    SystemMessage(content='you are a geography expert. and you only reply in korean.'),
    AIMessage(content='무엇이 궁금하신가요? 저는 지리학 박사입니다!'),
    HumanMessage(content='한국과 일본의 지리적 차이에 대해 알려줘.')
]

# Pass a message sequence to the model and return a message prediction.
# chat.predict_messages(messages)

In [39]:
### Prompt Templates
from langchain.prompts import PromptTemplate, ChatPromptTemplate 

# Load a prompt template from a template.
template = PromptTemplate.from_template('{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')

prompt = template.format(
country_a='korea',
country_b='japan')

# Pass a single string input to the model and return a string prediction.
# chat.predict(prompt) 

# Create a chat prompt template from a variety of message format그냥,, s.
template = ChatPromptTemplate.from_messages ([
    ('system', 'you are a geography expert. and you only reply in {language}.'),
    ('ai', '무엇이 궁금하신가요? 저는 {name}입니다!'),
    ('human', '{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')
])

prompt = template.format_messages(language="korean", name='배다롱', country_a='한국', country_b='일본')

# Pass a message sequence to the model and return a message prediction.
# chat.predict_messages(prompt) 

In [40]:
from langchain.schema import BaseOutputParser

# Output parsers help structure language model responses.
# My personal output parser
class CommaOutputParser(BaseOutputParser):
    # Parse a single string model output into some structure.
    def parse(self,text): 
        items= text.strip().split(',')
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
# p.parse("hello, how, are, you?")    
# ['hello', 'how', 'are', 'you?']

In [41]:

template = ChatPromptTemplate.from_messages([
    ("system", "you are a list generating machine. everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. do not reply with anything else."),
    ("human", "{question}"),
])

# general way
# prompt = template.format_messages(max_items=10, question = 'what are the colors?')
# results = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(results.content)

# LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. 
# RunnableSequence을 사용하여 여러 Runnable을 조합하고 체인을 구성할 수 있습니다. 
chain = template | chat | CommaOutputParser()
# chain.invoke({"max_items":5, "question": "flowers"})

In [42]:
# Chaining Chains
chef_template = ChatPromptTemplate.from_messages([
    ("system", "you are a world-class international chef. you create easy to follow recipies for any typeof cuisins with easy to find ingredients."),
    ("human", "I want to cook {cuision} food."),
])

chef_chain = chef_template | chat 

In [43]:
veg_chef_template = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_template | chat

In [44]:
final_chain = {"recipe": chef_chain} | veg_chain
# final_chain.invoke({'cuision':'indian'})

In [47]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate 

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])  

examples = [
    {
    "question": "What do you know about France?",
    "answer": """
    Here is what I know:
    Capital: Paris
    Language: French
    Food: Wine and Cheese
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Italy?",
    "answer": """
    I know this:
    Capital: Rome
    Language: Italian
    Food: Pizza and Pasta
    Currency: Euro
    """,
    },
    {
    "question": "What do you know about Greece?",
    "answer": """
    I know this:
    Capital: Athens
    Language: Greek
    Food: Souvlaki and Feta Cheese
    Currency: Euro
    """,
    }
]

examples_template = """
    Human: {question}
    AI: {answer}
"""

example_prompt = PromptTemplate.from_template(examples_template)

# Prompt template that contains few shot examples.
prompt = FewShotPromptTemplate(example_prompt=example_prompt, examples=examples, suffix="Human: What do you know about {country}", input_variables=["country"])

prompt.format(country="Korea") # validation



'\n    Human: What do you know about France?\n    AI: \n    Here is what I know:\n    Capital: Paris\n    Language: French\n    Food: Wine and Cheese\n    Currency: Euro\n    \n\n\n\n    Human: What do you know about Italy?\n    AI: \n    I know this:\n    Capital: Rome\n    Language: Italian\n    Food: Pizza and Pasta\n    Currency: Euro\n    \n\n\n\n    Human: What do you know about Greece?\n    AI: \n    I know this:\n    Capital: Athens\n    Language: Greek\n    Food: Souvlaki and Feta Cheese\n    Currency: Euro\n    \n\n\nHuman: What do you know about Korea'